In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
import logging
import subprocess
from copy import deepcopy
from time import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdDistGeom import EmbedMultipleConfs
from linux_qm.src.util import load_smiles3D, _create_tmp_dir, draw3Dconfs, set_atom_positions
from linux_qm.qm.crest.crest import conformer_pipeline
import pandas as pd

In [21]:
smi = 'COC1CN(C)C1'
# smi = 'CO'


# mol = load_smiles3D(smi)
# EmbedMultipleConfs(mol, numConfs=3)

mol = conformer_pipeline(smi, screen=False)

# Get the conformers

conformers = mol.GetConformers()

# Print the number of conformers generated
print(f"Number of conformers generated: {len(conformers)}")
print("Energies:", [c.GetProp('energy') for c in conformers])

[26-06-2024 02:13:44] [INFO] Conformer generation pipeline


Number of conformers generated: 4
Energies: ['-2.6892239400000002', '-2.6879368600000002', '-2.68740596', '-2.68664653']


In [22]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, autoalign=False)

(3, 2, 6, 4)
num conformers 4


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
import os
os.getcwd()

'/home/ergot/linux_qm/notebooks'

In [9]:
logging.getLogger().setLevel(logging.DEBUG)

In [10]:
from linux_qm.qm.orca.orca import OrcaDriver
orca = OrcaDriver()

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'XTB2'
orca.geometry_optimization(conf)

orca.options['n_jobs'] = 1
orca.options['method'] = 'HF-3c'
# orca.options['method'] = ''
data = orca.single_point(conf, calc_npa=True)

[26-06-2024 01:56:07] [INFO] Method: XTB2
[26-06-2024 01:56:07] [DEBUG] ORCA INPUT:
!XTB2 OPT  
%geom MaxIter 100 end
%maxcore 2000
*xyz 0 1
C            2.49009597          0.52056894          0.10790894
O            1.33946624          0.16581571         -0.61907911
C            0.50519056         -0.74537539          0.10022886
C           -0.34923328         -0.07900651          1.19307096
N           -1.53057685         -0.13088972          0.28018288
C           -1.91285336          1.13296735         -0.35686601
C           -0.80729925         -1.06641919         -0.63533616
H            3.07424903          1.22757048         -0.48197263
H            3.09292696         -0.36745586          0.30117526
H            2.21105321          0.98770570          1.05248281
H            1.06978879         -1.63381109          0.42724156
H           -0.05003605          0.94861624          1.42730621
H           -0.50105862         -0.68185050          2.09762347
H           -2.67947963    

In [11]:
data.metadata['input_file_name']

'task.inp'

In [12]:
for k,v in data.atomcharges.items():
    print(k)
    print(v)

mulliken
[-0.170527 -0.437291  0.066179 -0.152182 -0.323463 -0.259003 -0.142218
  0.14283   0.125367  0.124898  0.131906  0.126472  0.126085  0.132139
  0.131237  0.11176   0.130735  0.135075]
lowdin
[-0.076192 -0.319472  0.08039  -0.089614 -0.215432 -0.155919 -0.076896
  0.089709  0.074511  0.073951  0.074585  0.075753  0.074543  0.083157
  0.082477  0.064276  0.07803   0.082144]
npa
[-0.08843  -0.402448  0.129198 -0.112927 -0.27261  -0.207117 -0.101925
  0.107137  0.09112   0.090389  0.085853  0.094241  0.091768  0.106263
  0.105492  0.086693  0.09572   0.101583]


In [13]:
orca.geometry_optimization(conf)

[26-06-2024 01:56:17] [INFO] Method: HF-3c
[26-06-2024 01:56:17] [DEBUG] ORCA INPUT:
!HF-3c OPT  
%geom MaxIter 100 end
%maxcore 2000
*xyz 0 1
C            2.81541200          0.13577100          0.26563500
O            1.70521000         -0.03520500         -0.58017300
C            0.59693100         -0.59627600          0.06464600
C           -0.26543400          0.33350700          0.94221300
N           -1.45572700         -0.13684500          0.22928700
C           -2.43481100          0.84582700         -0.15439800
C           -0.63767300         -0.73008500         -0.83192800
H            3.60969500          0.55530400         -0.34836800
H            3.14332600         -0.82367500          0.68455200
H            2.59030000          0.82081800          1.09114400
H            0.87265000         -1.52792600          0.57596400
H           -0.02996700          1.38953300          0.76048400
H           -0.29624400          0.13599000          2.01471300
H           -3.20575700  

In [19]:
%%time

logging.getLogger().setLevel(logging.INFO)
orca.options['solvent'] = 'DMSO'
orca.options['n_jobs'] = 8

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-31G'
orca.geometry_optimization(conf)

[26-06-2024 02:01:06] [INFO] Method: M062X 6-31G
[26-06-2024 02:03:14] [INFO] Identified logfile to be in ORCA format
[26-06-2024 02:03:14] [INFO] Success: True
[26-06-2024 02:03:14] [INFO] Num Iter: 17
[26-06-2024 02:03:14] [INFO] Elapsed Time: 128.0s


CPU times: user 130 ms, sys: 7.24 ms, total: 137 ms
Wall time: 2min 8s


In [18]:
%%time

logging.getLogger().setLevel(logging.INFO)
orca.options['solvent'] = 'DMSO'

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-311++G(d,p) NMR'
orca.geometry_optimization(conf)

[26-06-2024 01:59:12] [INFO] Method: M062X 6-311++G(d,p) NMR


KeyboardInterrupt: 

In [17]:
%%time

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-31G'
orca.geometry_optimization(conf)

orca.options['method'] = 'M062X 6-311++G(d,p) NMR'
orca.geometry_optimization(conf)

[26-06-2024 01:58:59] [INFO] Method: M062X 6-31G
[26-06-2024 01:58:59] [INFO] Identified logfile to be in ORCA format
[26-06-2024 01:58:59] [INFO] Method: M062X 6-311++G(d,p) NMR
[26-06-2024 01:58:59] [INFO] Identified logfile to be in ORCA format


CPU times: user 8.25 ms, sys: 9.36 ms, total: 17.6 ms
Wall time: 119 ms


In [20]:
%%time

conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVDZ'
orca.geometry_optimization(conf)

[26-06-2024 02:04:17] [INFO] Method: M062X cc-pVDZ NMR
[26-06-2024 02:09:21] [INFO] Identified logfile to be in ORCA format
[26-06-2024 02:09:21] [INFO] Success: True
[26-06-2024 02:09:21] [INFO] Num Iter: 22
[26-06-2024 02:09:21] [INFO] Elapsed Time: 303.9s


CPU times: user 188 ms, sys: 32.9 ms, total: 221 ms
Wall time: 5min 3s


In [12]:
%%time
conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVTZ'
orca.geometry_optimization(conf)

[24-11-2023 02:38:10] [INFO] Geometry optimization
[24-11-2023 02:38:10] [INFO] Method: M062X cc-pVTZ NMR
[24-11-2023 02:41:28] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:41:28] [INFO] Success: True
[24-11-2023 02:41:28] [INFO] Num Iter: 9
[24-11-2023 02:41:28] [INFO] Elapsed Time: 198.2s


CPU times: user 175 ms, sys: 49.4 ms, total: 225 ms
Wall time: 3min 18s


In [13]:
%%time
conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X aug-cc-pVTZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 02:41:28] [INFO] Geometry optimization
[24-11-2023 02:41:28] [INFO] Method: M062X aug-cc-pVTZ NMR
[24-11-2023 02:50:45] [INFO] Identified logfile to be in ORCA format
[24-11-2023 02:50:45] [INFO] Success: True
[24-11-2023 02:50:45] [INFO] Num Iter: 9
[24-11-2023 02:50:45] [INFO] Elapsed Time: 556.3s


CPU times: user 195 ms, sys: 80.9 ms, total: 276 ms
Wall time: 9min 16s


In [17]:
%%time

conf = deepcopy(mol).GetConformer()
orca.options['method'] = 'M062X cc-pVDZ'
orca.geometry_optimization(conf)

orca.options['method'] = 'M062X aug-cc-pVTZ NMR'
orca.geometry_optimization(conf)

[24-11-2023 03:03:21] [INFO] Geometry optimization
[24-11-2023 03:03:21] [INFO] Method: M062X cc-pVDZ
[24-11-2023 03:04:42] [INFO] Identified logfile to be in ORCA format
[24-11-2023 03:04:42] [INFO] Success: True
[24-11-2023 03:04:42] [INFO] Num Iter: 9
[24-11-2023 03:04:42] [INFO] Elapsed Time: 80.5s
[24-11-2023 03:04:42] [INFO] Geometry optimization
[24-11-2023 03:04:42] [INFO] Method: M062X aug-cc-pVTZ NMR
[24-11-2023 03:10:10] [INFO] Identified logfile to be in ORCA format
[24-11-2023 03:10:10] [INFO] Success: True
[24-11-2023 03:10:10] [INFO] Num Iter: 5
[24-11-2023 03:10:10] [INFO] Elapsed Time: 328.7s


CPU times: user 234 ms, sys: 122 ms, total: 356 ms
Wall time: 6min 49s


In [15]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, confIds=[0], autoalign=False)

(3, 2, 6, 4)
num conformers 4


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [16]:
data.scfvalues

NameError: name 'data' is not defined

In [ ]:
data.scftargets

In [ ]:
# NMR shielding
for k,v in data.nmrtensors.items():
    print(mol.GetAtomWithIdx(k).GetSymbol(), k, v['isotropic'])

In [ ]:
dir(data)

In [ ]:
print(data.metadata['cpu_time'][0])

In [ ]:
len(data.converged_geometries)

In [ ]:
dir(data)

In [ ]:
data.optdone

In [ ]:
data.metadata

In [ ]:
data.new_geometries

In [ ]:
# HOMO LUMO
homo = data.moenergies[0][data.homos[0]]
lumo = data.moenergies[0][data.homos[0] + 1]
homo, lumo

In [ ]:
print(data.metadata['cpu_time'][0])

In [ ]:
data.natom, data.nmo

In [ ]:
data.metadata

In [ ]:
data.scfenergies